In [ ]:
import requests
from faker import Faker
import ujson as json

import pandas as pd
import requests
from random import randint

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import Search, DocType, Date, Integer, Keyword, Text
from datetime import datetime
from elasticsearch_dsl.connections import connections
import pandas as pd
import requests
from random import randint


In [ ]:
ES_HOST = 'http://ec2-34-205-15-150.compute-1.amazonaws.com:9200'
INDEX = "eda"
DOC_TYPE="user"
es = Elasticsearch(ES_HOST)

In [24]:
fake = Faker()
#Number of records to be generated
j = 0

#An empty list
actions = []

#Bgeinning loop
while (j <= 2005):
   
    action = {
        "_index": INDEX,
        '_op_type': 'create',
        "_type": DOC_TYPE,
        "_id": fake.uuid4(),
        "_source": {
            "name": fake.name(),
            "email_unsubscribe": fake.boolean(),
            "fav_color":fake.color_name(),
            "revenue" : randint(100,9567),
            "city": fake.city(), 
            "country": fake.country(),
            "address": fake.address(),
            "email" : fake.safe_email(),
            "signup_date" : fake.date_time_this_year(),
            "job": fake.job(),
            "r_score": int(fake.numerify('#')),
            "f_score": int(fake.numerify('#')),
            "m_score": int(fake.numerify('#')),
            "rfm_score": int("r_score") + int("f_score") + int("m_score"),
            "revenue": randint(100,9567),
            "avg_revenue_per_month": randint(100,9567) / int(randint(1,12)),
            "num_of_orders": (revenue / avg_revenue_per_month),
            "discount_percentage": int(randint(0,30)),
            "total_discount_revenue": revenue * discount_percentage /100,
            "lifecycle": int(randint(1,10))
            }
        }
    
    actions.append(action)
    j += 1
    

    while (len(actions) > 5000):
        helpers.bulk(es, actions)
        del actions[0:len(actions)]
        print("5000 more records inserted...")
        # Empty the actions list after 10,000 times
        break

# Some data is remaining since the loop has gone when len(actions) is less than 10,000
if (len(actions) > 0):
    helpers.bulk(es, actions)
    del actions[0:len(actions)]
print("All records inserted")
   

All records inserted


In [30]:
# Initialize the scroll
page = es.search(
  index = INDEX,
  doc_type = DOC_TYPE,
  scroll = '2000',
  search_type = 'scan',
  size = 10,
  body = {
    # Your query's body
    })
sid = page['_scroll_id']
scroll_size = page['hits']['total']
  
  # Start scrolling
while (scroll_size > 0):
    print("Scrolling...")
    page = es.scroll(scroll_id = sid, scroll = '2m')
    # Update the scroll ID
    sid = page['_scroll_id']
    # Get the number of results that we returned in the last scroll
    scroll_size = len(page['hits']['hits'])
    print("scroll size: " + str(scroll_size))
    # Do something with the obtained page

GET http://34.205.15.150:9200/umbrellacorp/user/_search?search_type=scan&size=10&scroll=2000 [status:400 request:0.003s]


RequestError: TransportError(400, 'illegal_argument_exception', 'No search type for [scan]')